<a href="https://colab.research.google.com/github/AI-ML-DL-EXPERT/Next-Word-Predictor/blob/main/Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import string

import pickle

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def load_doc(filename):
    # Open the file
    file = open(filename, "r")

    # Read all text
    text = file.read()

    # Close the file
    file.close()

    return text

In [ ]:
# Load Document
filename = "/content/drive/MyDrive/Deep_Learning_Datasets/republic_clean.txt"

doc = load_doc(filename)

print(doc[: 200])

﻿BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in wh


In [ ]:
# Turn a doc into clean tokens
def clean_doc(doc):
    # Replace "--" with a space " "
    doc = doc.replace("--", " ")

    # split into tokens by white space
    tokens = doc.split()

    # Remove punctuation from each token
    """The `str.maketrans("", "", string.punctuation)` function creates a
    translation table that can be used to remove punctuation from a string.
     It takes three arguments: the characters to be replaced, the characters
     to replace them with, and the characters to delete. In this case, it
     removes all punctuation from the string."""
    table = str.maketrans("", "", string.punctuation)

    tokens = [w.translate(table) for w in tokens]

    # Remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]

    # Make lower case
    tokens = [word.lower() for word in tokens]

    return tokens

In [ ]:
# Clean Document
tokens = clean_doc(doc)

print(tokens[: 200])
print("Total tokens: ", len(tokens))
print("Unique tokens: ", len(set(tokens)))

['i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid', 'us', '

In [ ]:
# Save Clean Text

# Organize into sequence of tokens
length = 50 + 1
sequences = list()

for i in range(length, len(tokens)):
    # Select sequence of tokens
    seq = tokens[i - length: i]

    # Convert into a line

    line = ' '.join(seq)

    # Store
    sequences.append(line)

print("Total sequences: ", len(sequences))

sequences[: 10]

Total sequences:  118632


['i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted',
 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with',
 'went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with the',
 'down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in wha

In [ ]:
# Save tokens to file, one dialog per line

def save_doc(lines, filename):
    data = "\n".join(lines)
    file = open(filename, "w")
    file.write(data)
    file.close()

In [ ]:
# Save sequences to file

out_filename = "/content/drive/MyDrive/Deep_Learning_Datasets/republic_sequences.txt"

save_doc(sequences, out_filename)

## Load Sequences

In [ ]:
# Load the doc into memory

in_filename = "/content/drive/MyDrive/Deep_Learning_Datasets/republic_sequences.txt"

doc = load_doc(in_filename)

lines = doc.split("\n")

In [ ]:
lines[:5]

['i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted',
 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with',
 'went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with the',
 'down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in wha

## Encode Sequences

In [ ]:
# Integer encode sequences of words

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

vocab_size

7410

In [ ]:
# Separate into input and output

sequences = np.array(sequences)

x, y = sequences[:, : -1], sequences[:, -1]

y = to_categorical(y, num_classes = vocab_size)

seq_length = x.shape[1]

In [ ]:
x.shape

(118632, 50)

In [ ]:
y.shape

(118632, 7410)

## Fit Model

In [ ]:
# Define Model

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length= seq_length))
model.add(LSTM(100, return_sequences= True))
model.add(LSTM(100))
model.add(Dense(100, activation = "relu"))
model.add(Dense(vocab_size, activation = "softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            370500    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7410)              748410    
                                                                 
Total params: 1269810 (4.84 MB)
Trainable params: 1269810 (4.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile Model

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Fit model

history = model.fit(x, y, batch_size= 128, epochs= 100)

Epoch 1/100
927/927 [==============================] - 51s 50ms/step - loss: 6.1504 - accuracy: 0.0743
Epoch 2/100
927/927 [==============================] - 20s 22ms/step - loss: 5.6793 - accuracy: 0.1081
Epoch 3/100
927/927 [==============================] - 17s 19ms/step - loss: 5.4291 - accuracy: 0.1319
Epoch 4/100
927/927 [==============================] - 16s 17ms/step - loss: 5.2784 - accuracy: 0.1442
Epoch 5/100
927/927 [==============================] - 16s 17ms/step - loss: 5.1678 - accuracy: 0.1521
Epoch 6/100
927/927 [==============================] - 16s 18ms/step - loss: 5.0751 - accuracy: 0.1583
Epoch 7/100
927/927 [==============================] - 15s 16ms/step - loss: 4.9938 - accuracy: 0.1639
Epoch 8/100
927/927 [==============================] - 15s 16ms/step - loss: 4.9182 - accuracy: 0.1695
Epoch 9/100
927/927 [==============================] - 15s 16ms/step - loss: 4.8437 - accuracy: 0.1738
Epoch 10/100
927/927 [==============================] - 16s 17ms/step - l

## Save the model

In [ ]:
# Save the model to file

model.save("model.h5")

# Save the tokenizer

pickle.dump(tokenizer, open("tokenizer.pkl", "wb"))

## Load Model

In [ ]:
# Load the model

loadModel_For_testing = keras.models.load_model("/content/model.h5")

In [ ]:
# Load the tokenizer

tokenizer = pickle.load(open("tokenizer.pkl", "rb"))

## Generate Text

In [ ]:
# Select a seed text

seed_text = lines[np.random.randint(0, len(lines))]

print(seed_text + "\n")

acquisition of knowledge of any kind bodily exercise when compulsory does no harm to the body but knowledge which is acquired under compulsion obtains no hold on the mind very true then my good friend i said do not use compulsion but let early education be a sort of amusement you



In [ ]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]

In [ ]:
encoded

[1157,
 3,
 104,
 3,
 45,
 211,
 492,
 982,
 40,
 6382,
 161,
 47,
 509,
 4,
 1,
 171,
 22,
 104,
 13,
 5,
 1050,
 180,
 2805,
 3125,
 47,
 489,
 65,
 1,
 159,
 67,
 38,
 31,
 92,
 48,
 182,
 11,
 19,
 41,
 12,
 172,
 2805,
 22,
 86,
 1376,
 245,
 10,
 8,
 119,
 3,
 1534,
 15]

In [ ]:
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Select a seed text
text = lines[np.random.randint(0, len(lines))]


for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=seq_length, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 [==============================] - 1s 693ms/step
principles about justice and honour which were taught us in childhood and under their parental authority we have been brought up obeying and honouring them that is true there are also opposite maxims and habits of pleasure which flatter and attract the soul but do not influence those of us who are
1/1 [==============================] - 0s 23ms/step
principles about justice and honour which were taught us in childhood and under their parental authority we have been brought up obeying and honouring them that is true there are also opposite maxims and habits of pleasure which flatter and attract the soul but do not influence those of us who are not
1/1 [==============================] - 0s 21ms/step
principles about justice and honour which were taught us in childhood and under their parental authority we have been brought up obeying and honouring them that is true there are also opposite maxims and habits of pleasure which flatter and 